# napari animation
Napari is great to visualize images. But sometimes, we want to save images or movies. You can easily script napari to put the settings just as you want, take images or even record movies of just the positions you are interested in.

Using the napari screenshot functionality and the napari_animation plugin, we explore those options here. The napari_animation plugin was one of the libraries you installed in the beginning.

In [ ]:
import napari
from napari_animation import Animation, AnimationWidget, easing
from pathlib import Path
import imageio

In [ ]:
viewer = napari.Viewer()

In [ ]:
image_folder = Path('/path/to/folder')

# For example:
image_folder = Path('/Users/joel/Desktop/TIF_OVR_MIP/')
img_filename = '220323SB050AAB_220326_215615_B02_T0001F001L01A01Z01C01.tif'
img = imageio.v2.imread(image_folder / img_filename)
layer1 = viewer.add_image(img, contrast_limits = [115, 2000], colormap='blue', blending='additive', scale=(0.325, 0.325))
img_filename_2 = '220323SB050AAB_220326_215615_B02_T0001F001L01A02Z01C02.tif'
img_2 = imageio.v2.imread(image_folder / img_filename_2)
layer2 = viewer.add_image(img_2, contrast_limits = [115, 1000], colormap='green', blending='additive', scale=(0.325, 0.325))
img_filename_3 = '220323SB050AAB_220326_215615_B02_T0001F001L01A03Z01C03.tif'
img_3 = imageio.v2.imread(image_folder / img_filename_3)
layer3 = viewer.add_image(img_3, contrast_limits = [115, 3000], colormap='red', blending='additive', scale=(0.325, 0.325))
viewer.scale_bar.visible=True
viewer.scale_bar.unit='um'

In [ ]:
img_sc = viewer.screenshot('napari_screenshot.png')

You may want to control the position & zoom level of the viewer before you take a screenshot. To do this, zoom to the region of interest. You can also set it programatically by changing `viewer.camera.zoom`and `viewer.camera.center`. 

In [ ]:
print(viewer.camera.zoom)
print(viewer.camera.center)

In [ ]:
viewer.camera.zoom = 3.39784349959661
viewer.camera.center = (0.0, 1741.0170395115192, 2336.2689073705806)
img_sc = viewer.screenshot('napari_screenshot_zoom_in_multicolor.png')

### Scripting screenshots 
If you are creating images for a figure, you may want to have the individual channels in black and white as well. This is something you can script in python

In [ ]:
for layer in viewer.layers:
    layer.colormap = 'gray'
layer2.visible = False
layer3.visible = False
img_sc = viewer.screenshot('napari_screenshot_zoom_in_channel1.png')
layer2.visible = True
layer1.visible = False
img_sc = viewer.screenshot('napari_screenshot_zoom_in_channel2.png')
layer3.visible = True
layer2.visible = False
img_sc = viewer.screenshot('napari_screenshot_zoom_in_channel3.png')

### Creating animations
Sometimes, a static image doesn't really represent your data well. You can use the `napari-animation` plugin to script movies.

The core part are still modifications to layer visibility, zoom & center positions. But now, we can add `keyframes` to the movements between specific positions and then use the animation plugin to create a movie of that

Some tips:
- If you want to fade in or out layers, use `layer.opacity = 0` for fade out and `layer.opacity = 1` to fade back in again. By setting keyframes, you can control the duration of the fade, instead of having a layer appear suddenly
- If you go to 3D, `viewer.dims.ndisplay = 3` switches into the 3D mode and changing `viewer.camera.angles` allows you to take extra control of camera parameters
- If you want the movement to feel smoother, use the `ease=easing.sine_ease_in_out` option in `animation.capture_keyframe()`.

In [ ]:
# Helper variable to determine general speed of the animation
time_units = 10

# Set initial position
viewer.camera.zoom = 0.2700881691843613
viewer.camera.center = (0.0, 1449.3101109305828, 2269.6402831261435)
layer3.visible = True
layer2.visible = True
layer1.visible = True
layer1.colormap = 'blue'
layer2.colormap = 'green'
layer3.colormap = 'red'


# Initialize the animation object
animation = Animation(viewer)

animation.capture_keyframe(steps=1*time_units)

# Zoom in to a new location
viewer.camera.zoom = 3.39784349959661
viewer.camera.center = (0.0, 1741.0170395115192, 2336.2689073705806)
animation.capture_keyframe(steps=8*time_units, ease=easing.sine_ease_in_out)
animation.capture_keyframe(steps=6*time_units)

animation.animate('napari_zoomin_movie.mp4', canvas_only=True, fps=30)

### Creating animations in 3D
To finish, let's have a look at some making some movies in 3D. We load the 3D data from the intro notebook again and make some movies with this. For this, close the existing viewer and open a fresh one.

In [ ]:
viewer = napari.Viewer()

In [ ]:
path_3D = Path('/Users/joel/Desktop/3D_Organoid/object_17')
channel1_3D_name = '220120CS001AAA72h_220206_161008_B04_T0001F001L01A04Z01C01O00017_TIF-OVR.tif'
channel1_3D = imageio.v2.volread(path_3D / channel1_3D_name)

channel2_3D_name = '220120CS001AAA72h_220206_161008_B04_T0001F001L01A04Z01C02O00017_TIF-OVR.tif'
channel2_3D = imageio.v2.volread(path_3D / channel2_3D_name)

channel3_3D_name = '220120CS001AAA72h_220206_161008_B04_T0001F001L01A04Z01C03O00017_TIF-OVR.tif'
channel3_3D = imageio.v2.volread(path_3D / channel3_3D_name)

channel4_3D_name = '220120CS001AAA72h_220206_161008_B04_T0001F001L01A04Z01C04O00017_TIF-OVR.tif'
channel4_3D = imageio.v2.volread(path_3D / channel4_3D_name)

In [ ]:
viewer.add_image(channel1_3D, contrast_limits = [115, 4000], colormap='blue', blending='additive', scale=(1, 0.1625, 0.1625))
viewer.add_image(channel2_3D, contrast_limits = [115, 4000], colormap='green', blending='additive', scale=(1, 0.1625, 0.1625))
viewer.add_image(channel3_3D, contrast_limits = [115, 1500], colormap='red', blending='additive', scale=(1, 0.1625, 0.1625))
viewer.add_image(channel4_3D, contrast_limits = [115, 3000], colormap='gray', blending='additive', scale=(1, 0.1625, 0.1625))
viewer.scale_bar.visible=True
viewer.scale_bar.unit='um'

First, let's switch to 3D, tune the display settings until we are happy and choose a good initial position. Besides zoom & center, we now also need to set the angles.

In [ ]:
print(viewer.camera.angles)
print(viewer.camera.zoom)
print(viewer.camera.center)

In [ ]:
# Helper variable to determine general speed of the animation
time_units = 10

# Initial position
viewer.camera.angles = (-10.821890358627593, 3.677729843675152, 93.41792131964563)
viewer.camera.zoom = 0.5648973402258087
viewer.camera.center = (38.43063662848773, 22.389691790023555, 80.0512593110973)

# Initialize the animation object
animation = Animation(viewer)

animation.capture_keyframe(steps=1*time_units)

# Zoom in phase
viewer.camera.angles = (-10.821890358627593, 3.677729843675152, 93.41792131964563)
viewer.camera.zoom = 8.34282023107716
viewer.camera.center = (36.87404325003319, 33.08705235030531, 45.955187040892106)
animation.capture_keyframe(steps=5*time_units)

# Briefly stay in place
animation.capture_keyframe(steps=1*time_units)

# Tilt
viewer.camera.angles = (-9.272421966687192, 14.277887896896427, 136.97067422288384)
viewer.camera.zoom = 8.34282023107716
viewer.camera.center = (36.87404325003319, 33.08705235030531, 45.955187040892106)

animation.capture_keyframe(steps=3*time_units)

# Rotate
viewer.camera.angles = (-122.11136228333524, 64.88897936606122, 53.79418293404105)
viewer.camera.zoom = 8.34282023107716
viewer.camera.center = (36.87404325003319, 33.08705235030531, 45.955187040892106)
animation.capture_keyframe(steps=5*time_units)

# Zoom in further
viewer.camera.angles = (-136.7308378711137, 10.075769996745352, 84.46836504432848)
viewer.camera.zoom = 19.547334270103736
viewer.camera.center = (32.66551488526433, 32.46265761120795, 40.52731557593654)
animation.capture_keyframe(steps=4*time_units)

# Hold camera
animation.capture_keyframe(steps=2*time_units)
animation.animate('napari_3D_movie.mp4', canvas_only=True, fps=30)